<a href="https://colab.research.google.com/github/sultanardian/kuliah/blob/main/machine%20learning%20%26%20deep%20learning/ML_DL_A_Tugas_Akhir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project

## Predict the Plant-Nitrogen Deficiency

References:
1. Improving nitrogen assessment with an RGB camera across uncertain natural light from above-canopy measurements
Authors: Bayu Taruna Widjaja Putra, Peeyush Soni
Journal: Precision Agriculture
Publisher: SPRINGER; Q1/IF 4.454
DOI: https://doi.org/10.1007/s11119-019-09656-8

2. Dataset of chlorophyll content estimation of Coffea Canephora using Red and Near-Infrared consumer-grade camera
Authors: Bayu Taruna Widjaja Putra, Peeyush Soni
Journal: Data in Brief
Publisher: ELSEVIER; Cite Score / SJR: 0.7 / 0.341 /Q1
DOI: https://doi.org/10.1016/j.dib.2018.10.035
(Dataset for this Project is available in 2nd paper)

Dataset information:
1. 600 Images were taken using Modified camera with longpass filter 665nm
2. RGB bands, Vegetation indices(VIs) obtained using Camera (unsupervised) and Spectrometer (unsupervised) were provided

GIVEN: a. RGB Value (obtained from Modified camera)
       b. SPAD value
       c. N value
       d. VIs obtained from Modified Camera
ASKED: 1. Provide the solution using Machine Learning in estimating N status from RGB values (Use: KNN, NAIVE BAYES, DECISION TREE, RANDOM FORESTS, LOGISTICS REGRESSION, NEURAL NETWORKS)
       2. Compare the result with unsupervised VIs values
       
       

## Let's begin: prepare your data



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import HTML, display

# all_data = pd.ExcelFile('drive/MyDrive/Dataset/dataset.xls')
camera_data = pd.read_excel('drive/MyDrive/Dataset/dataset.xls', sheet_name=0)
camera_data.head()

,R,G,B,SPAD,Nitorgen,Category,Normalized R,Normalized G,Normalized B,SR B/R,NDVI (cam),Excess NIRB
0,148.250692,179.049049,207.690971,65.285056,3.441078,High,0.277109,0.334677,0.388214,59.440279,0.166994,88.082201
1,136.581287,168.443204,197.892366,68.834642,3.557860,High,0.271578,0.334933,0.393489,61.311079,0.183306,90.760241
2,138.817423,167.905846,195.125772,65.488399,3.447768,High,0.276612,0.334574,0.388814,56.308349,0.168617,83.528275
3,147.430079,178.955346,209.400353,64.500763,3.415275,High,0.275166,0.334005,0.390829,61.970274,0.173669,92.415281
4,142.520233,175.684288,207.991093,57.482832,3.184385,Normal,0.270850,0.333876,0.395273,65.470859,0.186787,97.777664


Make sure you use the optional parmaters in read_csv to convert missing data (indicated by a ?) into NaN, and to add the appropriate column names (BI_RADS, age, shape, margin, density, and severity):

Evaluate whether the data needs cleaning; your model is only as good as the data it's given. Hint: use describe() on the dataframe.

In [ ]:
camera_data.describe()

,R,G,B,SPAD,Nitorgen,Normalized R,Normalized G,Normalized B,SR B/R,NDVI (cam),Excess NIRB
count,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,150.500387,177.821762,200.125957,45.334192,2.784695,0.284715,0.336514,0.378771,49.625569,0.141727,71.929764
std,8.776726,8.271203,9.679605,12.712755,0.418250,0.006856,0.001338,0.007875,7.654312,0.021930,12.397594
min,118.056590,147.033575,172.239564,13.199405,1.727460,0.268136,0.332104,0.357375,27.828538,0.079318,39.932987
25%,145.097756,172.921318,193.341825,36.209840,2.484504,0.279907,0.335557,0.372858,43.971653,0.125584,62.593540
50%,150.255500,177.809798,199.244695,43.926998,2.738398,0.284966,0.336774,0.378066,49.183614,0.140520,70.690896
75%,155.743623,183.106732,207.016201,56.584216,3.154821,0.289631,0.337494,0.384683,55.934655,0.157602,82.172763
max,182.035192,205.115929,227.360392,68.939833,3.561321,0.305041,0.339400,0.399729,69.417563,0.197036,105.157736


There are quite a few missing values in the data set. Before we just drop every row that's missing data, let's make sure we don't bias our data in doing so. Does there appear to be any sort of correlation to what sort of data has missing fields? If there were, we'd have to try and go back and fill that data in.

In [ ]:
camera_data.loc[(camera_data['R'].isnull()) |
              (camera_data['G'].isnull()) |
              (camera_data['B'].isnull()) |
              (camera_data['SPAD'].isnull()) |
              #(camera_data['Nitrogen'].isnull()) |
              (camera_data['Category'].isnull())]

,R,G,B,SPAD,Nitorgen,Category,Normalized R,Normalized G,Normalized B,SR B/R,NDVI (cam),Excess NIRB


If the missing data seems randomly distributed, go ahead and drop rows with missing data. Hint: use dropna().

Next you'll need to convert the Pandas dataframes into numpy arrays that can be used by scikit_learn. Create an array that extracts only the feature data we want to work with (age, shape, margin, and density) and another array that contains the classes (severity). You'll also need an array of the feature name labels.

In [ ]:
all_features = camera_data[['R', 'G','B']].values

all_classes = camera_data['Category'].values

feature_names = ['R', 'G', 'B']

all_features

array([[148.25069215, 179.04904885, 207.6909708 ],
       [136.58128694, 168.44320449, 197.89236636],
       [138.81742342, 167.90584602, 195.12577215],
       ...,
       [156.52664047, 172.25072886, 184.3551781 ],
       [154.85355054, 179.95112782, 200.11854637],
       [151.54104915, 172.88054043, 189.08023713]])

Some of our models require the input data to be normalized, so go ahead and normalize the attribute data. Hint: use preprocessing.StandardScaler().

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
all_features_scaled = scaler.fit_transform(all_features)
all_features_scaled

array([[-2.56538976e-01,  1.48504480e-01,  7.82193711e-01],
       [-1.58723355e+00, -1.13482674e+00, -2.30944735e-01],
       [-1.33224072e+00, -1.19984834e+00, -5.17000058e-01],
       ...,
       [ 6.87190318e-01, -6.74107659e-01, -1.63063853e+00],
       [ 4.96403242e-01,  2.57658081e-01, -7.66185673e-04],
       [ 1.18669540e-01, -5.97899030e-01, -1.14208539e+00]])

In [ ]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

all_encoded_classes = LE.fit_transform(all_classes)

## Decision Trees

Before moving to K-Fold cross validation and random forests, start by creating a single train/test split of our data. Set aside 75% for training, and 25% for testing.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_features_scaled, all_encoded_classes, random_state = 1, test_size = 0.2)

In [ ]:
DTC = DecisionTreeClassifier()
DTC.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
predict = DTC.predict(x_test)

In [ ]:
accuracy_score(predict, y_test)

0.6916666666666667

# RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_features_scaled, all_encoded_classes, random_state = 1, test_size = 0.2)

In [ ]:
RFC = RandomForestClassifier()
RFC.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
predict = RFC.predict(x_test)

In [ ]:
accuracy_score(predict, y_test)

0.7833333333333333

Now try a RandomForestClassifier instead. Does it perform better?

## SVM

Next try using svm.SVC with a linear kernel. How does it compare to the decision tree?

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_features_scaled, all_encoded_classes, random_state = 1, test_size = 0.2)

In [ ]:
svc = SVC(kernel = 'linear')
svc.fit(x_test, y_test)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
predict = svc.predict(x_test)

In [ ]:
accuracy_score(predict, y_test)

0.8083333333333333

## KNN


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_features_scaled, all_encoded_classes, random_state = 1, test_size = 0.2)

In [ ]:
KNN = KNeighborsClassifier()
KNN.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
predict = KNN.predict(x_test)

In [ ]:
accuracy_score(predict, y_test)

0.7833333333333333

## Naive Bayes



In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_features_scaled, all_encoded_classes, random_state = 1, test_size = 0.2)

In [ ]:
GNB = GaussianNB()
GNB.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
predict = GNB.predict(x_test)

In [ ]:
accuracy_score(predict, y_test)

0.575

## Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_features_scaled, all_encoded_classes, random_state = 1, test_size = 0.2)

In [ ]:
LogReg = LogisticRegression()
LogReg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
predict = LogReg.predict(x_test)

In [ ]:
accuracy_score(predict, y_test)

0.8416666666666667